<a href="https://colab.research.google.com/github/francescopatane96/ESM2_experiments/blob/main/ESM2_classification_FINETUNED_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets

# Load finetuned model from huggingface

In [2]:
from transformers import TFEsmForSequenceClassification

model = TFEsmForSequenceClassification.from_pretrained("francescopatane/esm2m35finetunedcytmem", output_attentions=True)

All model checkpoint layers were used when initializing TFEsmForSequenceClassification.

All the layers of TFEsmForSequenceClassification were initialized from the model checkpoint at francescopatane/esm2m35finetunedcytmem.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFEsmForSequenceClassification for predictions without further training.


## Tokenizing the data

All inputs to neural nets must be numerical. The process of converting strings into numerical indices suitable for a neural net is called **tokenization**. For natural language this can be quite complex, as usually the network's vocabulary will not contain every possible word, which means the tokenizer must handle splitting rarer words into pieces, as well as all the complexities of capitalization and unicode characters and so on.

With proteins, however, things are very easy. In protein language models, each amino acid is converted to a single token. Every model on `transformers` comes with an associated `tokenizer` that handles tokenization for it, and protein language models are no different. Let's get our tokenizer!

In [3]:
from transformers import AutoTokenizer

model_for_tokenizer = "francescopatane/esm2m35finetunedcytmem"
tokenizer = AutoTokenizer.from_pretrained(model_for_tokenizer)



In [6]:
sequence_to_predict = ["MKGKNRSLFVLLVLLLLHKVNNVLLERTIETLLECKNEYVKGENGYKLAKGHHCVEEDNL\
ERWLQGTNERRSEENIKYKYGVTELKIKYAQMNGKRSSRILKESIYGAHNFGGNSYMEGK\
DGGDKTGEEKDGEHKTDSKTDNGKGANNLVMLDYETSSNGQPAGTLDNVLEFVTGHEGNS\
RKNSSNGGNPYDIDHKKTISSAIINHAFLQNTVMKNCNYKRKRRERDWDCNTKKDVCIPD\
RRYQLCMKELTNLVNNTDTNFHRDITFRKLYLKRKLIYDAAVEGDLLLKLNNYRYNKDFC\
KDIRWSLGDFGDIIMGTDMEGIGYSKVVENNLRSIFGTDEKAQQRRKQWWNESKAQIWTA\
MMYSVKKRLKGNFIWICKLNVAVNIEPQIYRWIREWGRDYVSELPTEVQKLKEKCDGKIN\
YTDKKVCKVPPCQNACKSYDQWITRKKNQWDVLSNKFISVKNAEKVQTAGIVTPYDILKQ\
ELDEFNEVAFENEINKRDGAYIELCVCSVEEAKKNTQEVVTNVDNAAKSQATNSNPISQP\
VDSSKAEKVPGDSTHGNVNSGQDSSTTGKAVTGDGQNGNQTPAESDVQRSDIAESVSAKN\
VDPQKSVSKRSDDTASVTGIAEAGKENLGASNSRPSESTVEANSPGDDTVNSASIPVVSG\
ENPLVTPYNGLRHSKDNSDSDGPAESMANPDSNSKGETGKGQDNDMAKATKDSSNSSDGT\
SSATGDTTDAVDREINKGVPEDRDKTVGSKDGGGEDNSANKDAATVVGEDRIRENSAGGS\
TNDRSKNDTEKNGASTPDSKQSEDATALSKTESLESTESGDRTTNDTTNSLENKNGGKEK\
DLQKHDFKSNDTPNEEPNSDQTTDAEGHDRDSIKNDKAERRKHMNKDTFTKNTNSHHLNS\
NNNLSNGKLDIKEYKYRDVKATREDIILMSSVRKCNNNISLEYCNSVEDKISSNTCSREK\
SKNLCCSISDFCLNYFDVYSYEYLSCMKKEFEDPSYKCFTKGGFKDKTYFAAAGALLILL\
LL"]

In [7]:
tokenized_sequence = tokenizer(sequence_to_predict)

# Create the dataset

In [8]:
from datasets import Dataset

dataset = Dataset.from_dict(tokenized_sequence)

# Create tensors

In [9]:
tf_set = model.prepare_tf_dataset(
    dataset,
    batch_size=1,
    shuffle=False,
    tokenizer=tokenizer
)

# Predict

In [10]:
pred = model.predict(tf_set)

1/1 [==============================] - 16s 16s/step


let's inspect the output of the transformer:


In [32]:
pred

TFSequenceClassifierOutput(loss=None, logits=array([[ 0.517693  , -0.58419484]], dtype=float32), hidden_states=None, attentions=(array([[[[1.56004615e-02, 2.64493357e-02, 5.29824803e-03, ...,
          5.05397948e-05, 5.33011007e-05, 2.14731444e-05],
         [2.85071075e-01, 6.05504289e-02, 3.84469749e-03, ...,
          2.72384136e-06, 2.92805476e-06, 3.44852606e-06],
         [5.94229288e-02, 1.79858282e-02, 2.85486667e-03, ...,
          1.97685022e-05, 2.10164053e-05, 1.32537552e-05],
         ...,
         [6.92576077e-06, 3.50313530e-05, 2.83285244e-05, ...,
          4.29991959e-03, 4.05267300e-03, 2.98468173e-02],
         [7.02489751e-06, 2.81917655e-05, 2.96174985e-05, ...,
          4.61050868e-03, 4.33330564e-03, 2.94599719e-02],
         [5.84588724e-06, 2.74680770e-05, 5.29700919e-05, ...,
          5.40852966e-03, 5.14116883e-03, 2.43747476e-02]],

        [[6.40799329e-02, 2.80860793e-02, 2.44987253e-02, ...,
          3.06727707e-05, 2.95288737e-05, 1.64420035e-05],
 

- loss: None
- logits: output values (not normalized) associated with the two possible labels (0 or 1). the higher logit value corresponds to relate label choosen by the model.
- hidden states: None
- attentions: a list of tensors that contains attention weights for every input token respect to every other input token. they can be used to find the relevance of input regions to determine the prediction.

# Save attentions from output

In [12]:
attentions = pred.attentions

In [33]:
A=attentions[-1][0][0]

In [21]:
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import matplotlib.pyplot as plt
import numpy as np

A=np.log(A)
plt.imshow(A)
plt.colorbar()

NameError: ignored

# Transformers interpret